In [ ]:
import sympy as sp

V_m, R, T, a, b = sp.symbols('V_m R T a b')
P = sp.symbols('P')

# Peng-Robinson equation of state
P_eq = (R * T / (V_m - b)) - (a / (V_m * (V_m + b) + b * (V_m - b)))

integral = sp.integrate(P_eq, V_m)

integral_result = sp.simplify(integral)
integral_result
